In [125]:
import pandas as pd
import numpy as np
from joblib import Parallel, delayed
from datetime import datetime 

import time

In [126]:
month="jul2023"

In [127]:
def loop(file_name):
    return pd.read_csv(f"{month}/rural/{file_name}",encoding='ISO-8859-1',on_bad_lines='skip')

In [128]:
def NonGOVT(df):
    govt = [30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,48,72,73,74,75,80,81,82,101,102,103,104,105,106,107,'80B','42A','41A','40A','H31'
           'H32','H33','H41','H42','H43','HV41','H13']
    govt = [str(x) for x in govt]
    df['gov_flag'] = df['SUPPLY_TYPE'].isin(govt)
    return df

In [129]:
def billable(df):
    bil = ['Live','First Bill Issued','New Connection','Temp.Disconnected']
    df['bilable_flag'] = df['BILLING_STATUS'].isin(bil)
    return df

In [130]:
def arrear(df):
#     df = df[df['TOTPAYAMT'] > 0]
    df['ARREAR_FILTER'] = None
    df.loc[df['AMOUNT'] < 5000,"ARREAR_FILTER"] = "<5k"
    df.loc[df['AMOUNT'] >= 5000,"ARREAR_FILTER"]= ">=5k"
#     df.loc[df['TOTPAYAMT'] <= 10000,"ARREAR_FILTER"] = "<=10k"
#     df.loc[((df['TOTPAYAMT'] > 10000) & (df['TOTPAYAMT'] <= 100000)),"ARREAR_FILTER"] = "10k-1L"
#     df.loc[((df['TOTPAYAMT'] > 100000) & (df['TOTPAYAMT'] <= 5000000)),"ARREAR_FILTER"] = "1L-50L"
#     df.loc[df['TOTPAYAMT'] > 5000000,"ARREAR_FILTER"] = "GRET_50L"
    
    return df
    

In [131]:
def CatLoad(df):
    df['CONTRACTLOAD'] = pd.to_numeric(df['CONTRACTLOAD'],errors="coerce")
    df['LOAD_FILTER'] = None
    df.loc[df['CONTRACTLOAD'] < 5,"LOAD_FILTER"] = "0-5kw"
    df.loc[((df['CONTRACTLOAD'] >= 5) & (df['CONTRACTLOAD'] < 10)),"LOAD_FILTER"] = "5-9kw"
    df.loc[df['CONTRACTLOAD'] >= 10,"LOAD_FILTER"] = "great than 10kw"
    return df

    

In [132]:
def faulty_Ageing(df):
    df['METER_FAULT_CNT'] = pd.to_numeric(df['METER_FAULT_CNT'],errors="coerce")
    df['faulty_Ageing'] = None
    df.loc[df['METER_FAULT_CNT'] < 2,"faulty_Ageing"] = "faulty<2"
    df.loc[((df['METER_FAULT_CNT'] >= 2) & (df['METER_FAULT_CNT'] < 3)),"faulty_Ageing"] = "faulty>=2 TO <3"
    df.loc[df['METER_FAULT_CNT'] >= 3,'METER_FAULT_CNT'] = "faulty >=3"
    return df

In [133]:
def date(df):
    df['LAST_PAID_DATE'] = pd.to_datetime(df['LAST_PAID_DATE'],format='%d-%m-%Y',errors = "coerce")
    df['SUPP_RELEASE_DATE'] = pd.to_datetime(df['SUPP_RELEASE_DATE'],format='%d-%b-%Y',errors = "coerce")
#     df['BILLDATE'] = pd.to_datetime(df['BILLDATE'],format="%d-%m-%Y %H:%M:%S",errors = "coerce")
    df['LAST_MU_RDG_DATE'] = pd.to_datetime(df['LAST_MU_RDG_DATE'],format="%d-%b-%y",errors = "coerce")
    df['KYC_CREATE_DATE'] = pd.to_datetime(df['KYC_CREATE_DATE'],format="%d-%b-%y",errors = "coerce")
    df['DISCONNECTION_DT'] = pd.to_datetime(df['DISCONNECTION_DT'],format='%d-%b-%y',errors = "coerce")
    df['DISCONNECTION_CREATE_DT'] = pd.to_datetime(df['DISCONNECTION_CREATE_DT'],format='%d-%b-%y',errors = "coerce")
    
    return df

In [134]:
def unpaid(df):
    df['unpaid_status'] = None
    df.loc[df['LAST_PAID_DATE'] < '01-01-2023',"unpaid_status"] = "unpaid"
    df.loc[pd.isnull(df['LAST_PAID_DATE']),"unpaid_status"] = "neverpaid"     #df[~pd.isnull(df['Courses'])]
    df.loc[df['LAST_PAID_DATE'] >= '01-01-2023',"unpaid_status"] = "paid"
    
    return df

In [135]:
import os

mylist = os.listdir(f"{month}/rural")

In [136]:
mylist

['GONDA_ZONE__0723_45.CSV',
 'GONDA_ZONE__0723_46.CSV',
 'GONDA_ZONE__0723_49.CSV',
 'GONDA_ZONE__0723_CG.CSV',
 'GONDA_ZONE__0723_GM.CSV',
 'GONDA_ZONE__0723_SW.CSV',
 'MD_REPORT_0723_GD.CSV',
 'MD_REPORT_0723_KG.CSV',
 'MD_REPORT_0723_NP.CSV',
 'MD_REPORT_0723_TP.CSV']

In [137]:
start = time.time()
df = []
df = Parallel(n_jobs=-1, verbose=10,prefer="threads")(delayed(loop)(file_name) for file_name in mylist)
df = pd.concat(df, ignore_index=True)

end = time.time()
print("CSV:", end - start)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
C:\Users\HP\AppData\Local\Temp\ipykernel_18652\2608124263.py:2: DtypeWarning: Columns (7,174,180) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(f"{month}/rural/{file_name}",encoding='ISO-8859-1',on_bad_lines='skip')
C:\Users\HP\AppData\Local\Temp\ipykernel_18652\2608124263.py:2: DtypeWarning: Columns (0,7,174,186,219) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(f"{month}/rural/{file_name}",encoding='ISO-8859-1',on_bad_lines='skip')
C:\Users\HP\AppData\Local\Temp\ipykernel_18652\2608124263.py:2: DtypeWarning: Columns (7,174,186,219) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(f"{month}/rural/{file_name}",encoding='ISO-8859-1',on_bad_lines='skip')
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:   25.5s remaining:   59.6s
C:\Users\HP\AppData\Local\Temp\i

CSV: 112.71801900863647


In [138]:
df['CONTRACTLOAD'] = pd.to_numeric(df['CONTRACTLOAD'],errors="coerce")
df['MD_KW'] = pd.to_numeric(df['MD_KW'],errors="coerce")

In [139]:
column =list(df.columns)

In [140]:
df['BILLING_STATUS'].unique()

array(['Live', 'Account Closed', 'Temp.Disconnected', 'First Bill Issued',
       'Bill Stopped', 'Dismantled', 'New Connection'], dtype=object)

In [141]:
df.DIVISION.unique()

array(['EDD-I GONDA', 'EDD BALRAMPUR', 'EDD-I-BAHRAICH',
       'EDD-III-GONDA-COLONELGANJ', 'EDD-IV-GONDA-MANKAPUR',
       'EDD SHRAWASTI', 'EDD-II GONDA', 'EDD-III-BAHRAICH-KAISERGANJ',
       'EDD-II-BAHRAICH-NANPARA', 'EDD-TULSIPUR'], dtype=object)

In [142]:
column

['BOOK_CODE',
 'DIVISION',
 'DIVISION_ID',
 'SUBDIVISION',
 'SUBSTATION',
 'FEEDER',
 'SUPPLY_TYPE',
 'SCNO',
 'OLDSCNO',
 'BILLDATE',
 'BILLNO',
 'BILLED_MONTHS',
 'BILLDUEDATE',
 'BILLDISCON',
 'ADDRESS1',
 'ADDRESS2',
 'ADDRESS3',
 'NAME',
 'OWNERSHIP',
 'PIN',
 'PHONE',
 'EMAIL',
 'BILLBASIS',
 'CONTRACTLOAD',
 'VOLSUPLY',
 'PF',
 'MTR_NUMBER',
 'METERED_KWH',
 'METERED_KVAH',
 'ADJ_UNITS',
 'BILLABLE_UNIT',
 'MAX_DEMAND',
 'ENRGYCHG',
 'MCG',
 'FIXEDCHG',
 'DEMANDCHG',
 'EXCESSDCHG',
 'EDCHG',
 'VOLSUPREB',
 'VOLSUPSURCHG',
 'CAPACITOR',
 'LPFSURCHARGE',
 'SOLARCAP',
 'SOLARREBATE',
 'PROVADJ',
 'TOTBILL',
 'SUBSIDY',
 'CURTOTAMT',
 'LATEPAYSUR',
 'OLDDUEAMT',
 'TOTPAYAMT',
 'AMOUNT',
 'AMTAFTRDUE',
 'AMTAFTRDIS',
 'CONSID',
 'BILLING_STATUS',
 'SUPP_RELEASE_DATE',
 'MF',
 'BILLID',
 'PREV_RDG_DATE',
 'PRST_RDG_DATE',
 'PREV_RDG_KWH',
 'PREV_RDG_KVAH',
 'PRST_RDG_KWH',
 'PRST_RDG_KVAH',
 'BILLED_CATEGORY',
 'APPROVE',
 'GRACEDATE',
 'ADDL_SURCHARGE',
 'FATHER_NAME',
 'CAT_CODE',
 

In [143]:
im_col = ['DIVISION',
          'SCNO',
          'ACCOUNTNO',

 'SUBDIVISION',
 'SUBSTATION',
 'FEEDER',
 'SUPPLY_TYPE',
          'CAT_CODE',
          'ADDRESS1',
          
 'ADDRESS2',
 'ADDRESS3',
 'NAME','PHONE', 'BILLBASIS','BILLDATE',
          'CONSUMPTION_UNIT',
          'BILLABLE_UNIT',
          
 'CONTRACTLOAD','MD_KW','TOTBILL','CURTOTAMT','TOTPAYAMT',
 'AMOUNT','CURYRARR',
 'CURYRLPS','BILLING_STATUS','SUPP_RELEASE_DATE','BILLED_CATEGORY'
]

In [144]:

df = df[im_col]
df['month'] = month
df = NonGOVT(df)
df = billable(df)

In [145]:
df1 = df[df['MD_KW'] > df['CONTRACTLOAD']]

In [146]:
df1.to_csv(f"load/rural/{month}.csv")

,DIVISION,SCNO,SUBDIVISION,SUBSTATION,FEEDER,SUPPLY_TYPE,CAT_CODE,ADDRESS1,ADDRESS2,ADDRESS3,...,MTR_STATUS,LAST_PAID_AMT,LAST_PAID_DATE,GOVT_FLAG,GOVT_CODE,GOVT_TYPE,PAYMENT_TYPE,AMOUNT_PAID_IN_BILLMONTH,SCHEME,OPR_FLAG
count,427693,427693,427693,405441,403244,427693,427693,397249,426218,111347,...,427693,427693,219293,427661,427661,427661,35728,427693,237456,427693
unique,4,427015,11,34,111,64,14,32257,60353,23694,...,9,23376,2259,54,54,4,8,5916,5,3
top,EDD-III-BAHRAICH-KAISERGANJ,SCNO,EDSD-I BAHRAICH NANPARA,FAKHARPUR,JALIM NAGAR,17,LMV-1,BAHRAICH,BAHRAICH,NANPARA,...,Normal,0,31-03-2021,NON-GOVERNMENT,NG,NG,COUNTER_PAYMENT,0,Saubhagya,Y
freq,206523,7,62090,47412,31089,349904,406885,44455,7773,5061,...,317145,195628,2303,391988,391988,391988,21285,365638,236687,374727


,DIVISION,SCNO,SUBDIVISION,SUBSTATION,FEEDER,SUPPLY_TYPE,CAT_CODE,ADDRESS1,ADDRESS2,ADDRESS3,...,LAST_PAID_AMT,LAST_PAID_DATE,GOVT_FLAG,GOVT_CODE,GOVT_TYPE,PAYMENT_TYPE,AMOUNT_PAID_IN_BILLMONTH,SCHEME,OPR_FLAG,gov_flag
0,EDD-II-BAHRAICH-NANPARA,NP10403055841,EDSD-I BAHRAICH NANPARA,SAHABA/BABAGANJ,BABAGANJ,17,LMV-1,SORAHIYA,BABAGANJ,NNP,...,0,NaN,GOVERNMENT,G,CG,NaN,0,NaN,Y,False
1,EDD-II-BAHRAICH-NANPARA,NP10527045041,EDSD-I BAHRAICH NANPARA,BANJARAN TADA (TAHSIL NANPARA),KAKRAHA,13,LMV-1,BAHRAICH,SAUKHAT,CHHOTI MASZID RUPAIDE,...,0,NaN,GOVERNMENT,G,CG,NaN,0,NaN,N,False
2,EDD-II-BAHRAICH-NANPARA,NP10563050910,EDSD-I BAHRAICH NANPARA,BANJARAN TADA (TAHSIL NANPARA),KAKRAHA,17,LMV-1,NaN,KESHWAPUR,NANPARA,...,0,NaN,GOVERNMENT,G,CG,NaN,0,NaN,Y,False
3,EDD-II-BAHRAICH-NANPARA,NP10402045026,EDSD-I BAHRAICH NANPARA,SAHABA/BABAGANJ,RUPAIDEEHA,17,LMV-1,BAHRAICH,HASMAT ALI,RUPAIDEHA,...,3160,07-01-2022,GOVERNMENT,G,CG,NaN,0,NaN,Y,False
4,EDD-II-BAHRAICH-NANPARA,NP10103305121,EDSD-I BAHRAICH NANPARA,NANPARA,MIHIPURWA,17,LMV-1,VITAHNIYA,NANPARA,NaN,...,0,NaN,Central Government,CG,CG,NaN,0,NaN,Y,False


,PHONE,CONTRACTLOAD,TOTBILL,CURTOTAMT,TOTPAYAMT,AMOUNT,ACCOUNTNO,LAST_PAID_AMT,AMOUNT_PAID_IN_BILLMONTH
count,3.738630e+05,420802.000000,4.208020e+05,4.208020e+05,4.208020e+05,4.208020e+05,4.208020e+05,4.208020e+05,4.208020e+05
mean,9.194817e+09,1.101328,5.092195e+02,5.092195e+02,1.233320e+04,1.228029e+04,7.395303e+11,1.561345e+03,2.073161e+02
std,1.272613e+10,9.614157,9.616938e+03,9.616938e+03,4.189756e+04,3.723896e+04,1.300285e+10,1.043153e+04,9.485972e+03
min,0.000000e+00,0.000000,-7.825320e+05,-7.825320e+05,-5.613352e+06,-5.613352e+06,7.316000e+11,0.000000e+00,0.000000e+00
25%,8.318045e+09,1.000000,1.093500e+02,1.093500e+02,7.946450e+02,7.900000e+02,7.318136e+11,0.000000e+00,0.000000e+00
50%,9.454203e+09,1.000000,2.394600e+02,2.394600e+02,6.559535e+03,6.554000e+03,7.319049e+11,0.000000e+00,0.000000e+00
75%,9.839558e+09,1.000000,6.362100e+02,6.362100e+02,1.125549e+04,1.124900e+04,7.616192e+11,1.278000e+03,0.000000e+00
max,1.000000e+12,4450.000000,4.169910e+06,4.169910e+06,1.063183e+07,7.925403e+06,7.617023e+11,4.169910e+06,4.169910e+06


In [56]:
# df = date(df)
# df = unpaid(df)
df = NonGOVT(df)
df = billable(df)
# df = faulty_Ageing(df)
# df['AMOUNT'] = pd.to_numeric(df['AMOUNT'],errors="coerce")
# df = arrear(df)
# df=CatLoad(df)

<ipython-input-22-419b918d01a9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ARREAR_FILTER'] = None
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [25]:
df.head()

,DIVISION,SCNO,SUBDIVISION,SUBSTATION,FEEDER,SUPPLY_TYPE,CAT_CODE,ADDRESS1,ADDRESS2,ADDRESS3,...,GOVT_FLAG,GOVT_CODE,GOVT_TYPE,PAYMENT_TYPE,AMOUNT_PAID_IN_BILLMONTH,SCHEME,OPR_FLAG,gov_flag,bilable_flag,ARREAR_FILTER
0,EDD-II-BAHRAICH-NANPARA,NP10403055841,EDSD-I BAHRAICH NANPARA,SAHABA/BABAGANJ,BABAGANJ,17,LMV-1,SORAHIYA,BABAGANJ,NNP,...,GOVERNMENT,G,CG,NaN,0,NaN,Y,False,True,10k-1L
3,EDD-II-BAHRAICH-NANPARA,NP10402045026,EDSD-I BAHRAICH NANPARA,SAHABA/BABAGANJ,RUPAIDEEHA,17,LMV-1,BAHRAICH,HASMAT ALI,RUPAIDEHA,...,GOVERNMENT,G,CG,NaN,0,NaN,Y,False,True,0-10k
4,EDD-II-BAHRAICH-NANPARA,NP10103305121,EDSD-I BAHRAICH NANPARA,NANPARA,MIHIPURWA,17,LMV-1,VITAHNIYA,NANPARA,NaN,...,Central Government,CG,CG,NaN,0,NaN,Y,False,True,0-10k
5,EDD-II-BAHRAICH-NANPARA,NP10417203506,EDSD-I BAHRAICH NANPARA,NaN,NaN,17,LMV-1,KALLYANPUR,TRILOKPUR,NaN,...,Central Government,CG,CG,NaN,0,Saubhagya,Y,False,True,0-10k
6,EDD-II-BAHRAICH-NANPARA,NP10417203711,EDSD-I BAHRAICH NANPARA,NaN,NaN,17,LMV-1,KALLYANPUR,TRILOKPUR,NaN,...,Central Government,CG,CG,NaN,0,Saubhagya,Y,False,True,10k-1L


,PHONE,CONTRACTLOAD,TOTBILL,CURTOTAMT,TOTPAYAMT,AMOUNT,ACCOUNTNO,LAST_PAID_AMT,AMOUNT_PAID_IN_BILLMONTH
count,3.431170e+05,370379.000000,3.703790e+05,3.703790e+05,3.703790e+05,3.703790e+05,3.703790e+05,3.703790e+05,3.703790e+05
mean,8.995727e+09,1.101048,6.286466e+02,6.286466e+02,1.387477e+04,1.386039e+04,7.388239e+11,1.690594e+03,2.686382e+02
std,6.617287e+09,10.147789,2.787247e+04,2.787247e+04,4.976595e+04,4.953997e+04,1.258189e+10,1.394016e+04,2.803506e+04
min,0.000000e+00,0.040000,-8.910830e+05,-8.910830e+05,-5.613442e+06,-5.613442e+06,7.316000e+11,0.000000e+00,0.000000e+00
25%,8.291664e+09,1.000000,1.294500e+02,1.294500e+02,2.429530e+03,2.425000e+03,7.318135e+11,0.000000e+00,0.000000e+00
50%,9.450258e+09,1.000000,3.188400e+02,3.188400e+02,7.307610e+03,7.306000e+03,7.319047e+11,1.000000e+02,0.000000e+00
75%,9.836537e+09,1.000000,6.343300e+02,6.343300e+02,1.185939e+04,1.185600e+04,7.322001e+11,1.526000e+03,0.000000e+00
max,1.000000e+12,4450.000000,1.513922e+07,1.513922e+07,1.512410e+07,1.513922e+07,7.617023e+11,5.164381e+06,1.498798e+07


In [17]:
df.dtypes

DIVISION                     object
SUBDIVISION                  object
SUBSTATION                   object
FEEDER                       object
SUPPLY_TYPE                  object
ADDRESS1                     object
ADDRESS2                     object
ADDRESS3                     object
NAME                         object
PHONE                       float64
BILLBASIS                    object
CONTRACTLOAD                float64
TOTBILL                     float64
CURTOTAMT                   float64
TOTPAYAMT                   float64
AMOUNT                      float64
BILLING_STATUS               object
SUPP_RELEASE_DATE            object
BILLED_CATEGORY              object
ACCOUNTNO                     int64
MTR_STATUS                   object
LAST_PAID_AMT               float64
LAST_PAID_DATE               object
GOVT_FLAG                    object
GOVT_CODE                    object
GOVT_TYPE                    object
PAYMENT_TYPE                 object
AMOUNT_PAID_IN_BILLMONTH    

0         30-06-2022
1         14-07-2022
2         21-06-2022
3         11-08-2022
4         30-08-2022
             ...    
425441    28-08-2022
425443    16-03-2020
425444    21-03-2021
425445           NaN
425446    19-12-2018
Name: LAST_PAID_DATE, Length: 370379, dtype: object

In [20]:
# df['LAST_PAID_DATE'] = pd.to_datetime(df.LAST_PAID_DATE)
# df['SUPP_RELEASE_DATE'] = pd.to_datetime(df.SUPP_RELEASE_DATE)

In [21]:
df.dtypes


DIVISION                     object
SUBDIVISION                  object
SUBSTATION                   object
FEEDER                       object
SUPPLY_TYPE                  object
ADDRESS1                     object
ADDRESS2                     object
ADDRESS3                     object
NAME                         object
PHONE                       float64
BILLBASIS                    object
CONTRACTLOAD                float64
TOTBILL                     float64
CURTOTAMT                   float64
TOTPAYAMT                   float64
AMOUNT                      float64
BILLING_STATUS               object
SUPP_RELEASE_DATE            object
BILLED_CATEGORY              object
ACCOUNTNO                     int64
MTR_STATUS                   object
LAST_PAID_AMT               float64
LAST_PAID_DATE               object
GOVT_FLAG                    object
GOVT_CODE                    object
GOVT_TYPE                    object
PAYMENT_TYPE                 object
AMOUNT_PAID_IN_BILLMONTH    

In [22]:
df.LAST_PAID_DATE.values

array(['02-06-2022', '28-01-2021', '12-09-2022', ..., nan, nan,
       '15-02-2020'], dtype=object)

In [133]:
# inner_join = pd.merge(left=df,right=df1,on ='ACCOUNTNO',how='right')

In [134]:
# inner_join

,DIVISION,BILLBASIS,SUBDIVISION,SUBSTATION,KYC_CREATE_DATE,DISCONNECTION_DT,DISCONNECTION_CREATE_DT,ACCOUNTNO,Sub-div,DISCONNECTION DONE_BY
0,EDD-III-BAHRAICH-KAISERGANJ,MU,EDSD-III-BAHRAICH-FAKHARPUR,FAKHARPUR_354923,12-09-22,NaN,NaN,731911815545,EDSD-III-BAHRAICH-,JE-DIST-SUNIL \tKUMAR
1,EDD-III-BAHRAICH-KAISERGANJ,MU,EDSD-III-BAHRAICH-FAKHARPUR,FAKHARPUR_354923,NaN,NaN,NaN,731904554728,EDSD-III-BAHRAICH-,JE-DIST-SUNIL \tKUMAR
2,EDD-III-BAHRAICH-KAISERGANJ,MU,EDSD-III-BAHRAICH-FAKHARPUR,TEJWAPUR_354923,NaN,NaN,NaN,731902044463,EDSD-III-BAHRAICH-,JE-DIST-SUNIL \tKUMAR
3,EDD-III-BAHRAICH-KAISERGANJ,MU,EDSD-III-BAHRAICH-FAKHARPUR,TEJWAPUR_354923,NaN,NaN,NaN,732001684168,EDSD-III-BAHRAICH-,JE-DIST-SUNIL \tKUMAR
4,EDD-III-BAHRAICH-KAISERGANJ,MU,EDSD-III-BAHRAICH-FAKHARPUR,FAKHARPUR_354923,NaN,NaN,NaN,731812748720,EDSD-III-BAHRAICH-,JE-DIST-SUNIL \tKUMAR
...,...,...,...,...,...,...,...,...,...,...
6357,EDD-TULSIPUR,NaN,EDSD-UTRAULA,UTRAULA TAHSIL_355133,NaN,NaN,NaN,732000629570,EDSD-UTRAULA,JE-DIST-Vijay Ranjan
6358,EDD-TULSIPUR,NaN,EDSD-REHRA,ACHALPUR CHAUDHARY_355133,NaN,NaN,NaN,731703564326,EDSD-REHRA,JE-DIST-Brijnandan\tyadav
6359,EDD-TULSIPUR,NaN,EDSD-REHRA,SUB-STATION REHRA_355133,NaN,NaN,NaN,731818412228,EDSD-REHRA,SDO-DIST-Raj Kumar Yadav
6360,EDD-TULSIPUR,NaN,EDSD-REHRA,SUB-STATION REHRA_355133,NaN,NaN,NaN,731901388754,EDSD-REHRA,JE-DIST-ANAY\tKUMAR


In [57]:
df1 = df[df["bilable_flag"] == True]

In [58]:
df1 = df1[df1['gov_flag'] == False]

In [59]:
start = time.time()
# df.to_csv("EDC_new.CSV")
df.to_csv("aprail2022/zone/Gonda.csv") 
df1.to_csv("aprail2022/zone/Gonda2.csv")
# df.to_csv("jan2023/Zone/NON_GONDA_jan.csv")

end = time.time()
print("CSV:", end - start)

CSV: 28.85196328163147


In [51]:
df1

,DIVISION,SCNO,ACCOUNTNO,SUBDIVISION,SUBSTATION,FEEDER,SUPPLY_TYPE,CAT_CODE,ADDRESS1,ADDRESS2,...,GOVT_CODE,GOVT_TYPE,PAYMENT_TYPE,AMOUNT_PAID_IN_BILLMONTH,SCHEME,OPR_FLAG,DISCONNECTION_DT,DISCONNECTION_CREATE_DT,gov_flag,bilable_flag
0,EDD-I GONDA,4541321101417,731703920424,EDSD-I GONDA,JAIL ROAD,UTRAULA,17,LMV-1,SONI KAPOOR,BABHNI KANOONGO,...,NG,NG,NaN,0.0,NaN,Y,NaN,NaN,False,True
1,EDD-I GONDA,4541422155288,731812669685,EDSD-I GONDA,JAIL ROAD,GONDA NORTH,17,LMV-1,"MADH NAGAR,MUSOULI",JHANJHARI,...,NG,NG,NaN,0.0,Saubhagya,Y,NaN,NaN,False,True
2,EDD-I GONDA,4531722002711,731816558144,EDSD-I GONDA,JAIL ROAD,MANKAPUR,17,LMV-1,MADHAIPUR,KHORHASA,...,NG,NG,NaN,0.0,Saubhagya,Y,NaN,NaN,False,True
3,EDD-I GONDA,4531922154976,731811719980,EDSD-I GONDA,JAIL ROAD,KHARGOO CHANDPUR,17,LMV-1,BHAYA PURWA MADHAIPUR,PASASPUR,...,NG,NG,NaN,0.0,Saubhagya,Y,NaN,NaN,False,True
5,EDD-I GONDA,4541321208060,731811148258,EDSD-I GONDA,JAIL ROAD,UTRAULA,17,LMV-1,LUVAI LUMBAI BARGAON KHAS,PANDRI KRIPAL GONDA,...,NG,NG,NaN,0.0,Saubhagya,Y,NaN,NaN,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1267192,EDD-TULSIPUR,TP24512412160,761626475409,EDSD-UTRAULA,UTRAULA TAHSIL (1X5MVA),TOWN NO.2,17,LMV-1,BALRAMPUR,R/O BANDIJOT,...,G,CG,NaN,0.0,NaN,Y,NaN,NaN,False,True
1267194,EDD-TULSIPUR,TP24516421285,731908409201,EDSD-UTRAULA,NaN,NaN,17,LMV-1,MUMTAJ DEEH,GAJPUR GRENT,...,NG,NG,NaN,0.0,NaN,Y,NaN,NaN,False,True
1267195,EDD-TULSIPUR,TP24716964705,731817101350,EDSD-UTRAULA,GAIDAS BUJURG,GAJPUR GRANT,17,LMV-1,THAKURAINDEEH BHARATPURGRINT,DAUTLATABAD GAINDASBUJURG,...,NG,NG,NaN,0.0,Saubhagya,Y,NaN,NaN,False,True
1267196,EDD-TULSIPUR,TP24516418571,761626531285,EDSD-UTRAULA,NaN,NaN,17,LMV-1,UTRAULA,HARI KRISHNA,...,NG,NG,CSC_PAYMENT,89.0,NaN,Y,NaN,NaN,False,True


In [24]:

# # import pandas
# import pandas as pd
   
# # read csv data
# df1 = pd.read_csv('Student_data.csv')
# df2 = pd.read_csv('Course_enrolled.csv')
   
# Left_join = pd.merge(df1, 
#                      df2, 
#                      on ='Name', 
#                      how ='left')
# Left_join

FileNotFoundError: [Errno 2] File Student_data.csv does not exist: 'Student_data.csv'